In [1]:
from tape import Ensemble
from tape.utils import ColumnMapper
from tape.analysis import calc_sf2
from tape.analysis.structure_function.base_argument_container import StructureFunctionArgumentContainer
from pyarrow import parquet
import pandas as pd
import dask.dataframe as dd
import numpy as np

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# Working with the TAPE Single Pixel Dataset

## Loading and Pre-Processing

In [2]:
# Initialize the Ensemble
data_path = "/data3/epyc/data3/hipscat/catalogs/tape_test/"
#parquet.read_schema(f"/data3/epyc/data3/hipscat/catalogs/tape_test/tape_test_sources/Norder=6/Dir=10000/Npix=17752.parquet", memory_map=True)  # <-- For inspection of the available columns

ens = Ensemble()
ens.client_info()

/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45096 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:45096/status,
Dashboard: http://127.0.0.1:45096/status,Workers: 12
Total threads: 96,Total memory: 188.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36104,Workers: 12
Dashboard: http://127.0.0.1:45096/status,Total threads: 96
Started: Just now,Total memory: 188.64 GiB
Comm: tcp://127.0.0.1:38787,Total threads: 8
Dashboard: http://127.0.0.1:38662/status,Memory: 15.72 GiB
Nanny: tcp://127.0.0.1:40060,


In [3]:
# Load the dataset into an Ensemble

col_map = ColumnMapper(id_col="_hipscat_index", 
                       time_col="mjd_ztf_source",
                       flux_col="mag_ztf_source", 
                       err_col="magerr_ztf_source",
                       band_col="band_ztf_source")

ens.from_hipscat(data_path,
                 source_subdir="tape_test_sources",
                 object_subdir="tape_test_obj",
                 column_mapper=col_map,
                 additional_cols=True,
                 sync_tables=True)


ens._object.head(5)

/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/dask/dataframe/core.py:5214: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/dask/dataframe/core.py:5214: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


,nobs_g,nobs_i,nobs_r,nobs_total,SDSS_NAME_dr16q_constant,PLATE_dr16q_constant,MJD_dr16q_constant,FIBERID_dr16q_constant,RA_dr16q_constant,DEC_dr16q_constant,...,zFlags_ps1_otmo,yMeanPSFMag_ps1_otmo,yMeanPSFMagErr_ps1_otmo,yFlags_ps1_otmo,Norder_ps1_otmo,Dir_ps1_otmo,Npix_ps1_otmo,_DIST,Norder,Dir
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
4996744218186612736,17,0,31,48,b'024311.71-010928.3',7337,56662,360,40.798814,-1.157879,...,114720,-999.000000,-999.000000,16416,5,0,4438,0.000015,5,0
4996744985459032064,71,2,76,149,b'024321.81-010812.1',6782,56572,285,40.840898,-1.136720,...,115000,-999.000000,-999.000000,16416,5,0,4438,0.000010,5,0
4996747488372195328,17,0,19,36,b'024315.98-010433.7',4240,55455,202,40.816592,-1.076055,...,115000,-999.000000,-999.000000,114720,5,0,4438,0.000017,5,0
4996749076859977728,85,3,82,170,b'024328.22-010352.7',6782,56572,298,40.867601,-1.064654,...,115000,-999.000000,-999.000000,114720,5,0,4438,0.000005,5,0
4996749735944519680,252,6,265,523,b'024341.05-010243.3',4240,55455,178,40.921044,-1.045386,...,115000,19.193501,0.039844,115000,5,0,4438,0.000016,5,0


In [4]:
# Numeric Band Conversions - Has to be done manually the moment

label_df = ens._source[["filterID_ps_source"]].replace([1.0,2.0,3.0,4.0,5.0], ["u","g","r","i","z"]).rename(columns={"filterID_ps_source": "band_ps_source"})
ens._source = dd.concat([ens._source, label_df], axis=1)

/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [5]:
# Convert Panstarrs fluxes to Magnitudes -- produces "mag_ps_source" and "mag_ps_source_err" columns
ens.convert_flux_to_mag("apFlux_ps_source", "zp_ps_source", "apFluxErr_ps_source", zp_form="mag", out_col_name="mag_ps_source")

In [6]:
# Coalescing - Need to combine survey specific columns (fluxes, times, etc)

ens.coalesce(["mjd_ztf_source", "obsTime_ps_source"], 
              "mvp_time", table="source", drop_inputs=True) # time
ens.coalesce(["mag_ztf_source", "mag_ps_source"], 
              "mvp_flux", table="source", drop_inputs=True) # flux
ens.coalesce(["magerr_ztf_source", "mag_ps_source_err"], 
              "mvp_error", table="source", drop_inputs=True) # Error
ens.coalesce(["band_ztf_source", "band_ps_source"], 
              "mvp_band", table="source", drop_inputs=True) # band, ps1 needs another filter table for translation?

#ens.coalesce(["mag_ztf_source", "apFlux_ps_source"], 
#              "mvp_flux", table="source", drop_inputs=True) # flux, potentially a mag/flux difference here
#ens.coalesce(["magerr_ztf_source", "apFluxErr_ps_source"], 
#              "mvp_error", table="source", drop_inputs=True) # Error, potentially a mag/flux difference here

In [7]:
ens.select(["mvp_time","mvp_flux","mvp_error","mvp_band","zp_ps_source"], table="source")
ens.compute("source").loc[4997869293954662400]

/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,zp_ps_source,mvp_time,mvp_flux,mvp_error,mvp_band
_hipscat_index,,,,,
4997869293954662400,NaN,58493.191720,20.966549,0.190953,g
4997869293954662400,NaN,58686.492020,19.881145,0.110847,g
4997869293954662400,NaN,58699.474040,20.917053,0.187413,g
4997869293954662400,NaN,58493.188040,21.220297,0.209098,g
4997869293954662400,NaN,58491.165140,21.221714,0.209200,g
...,...,...,...,...,...
4997869293954662400,24.487400,55479.503688,37.759749,0.058760,u
4997869293954662400,24.570801,56907.528867,36.348323,0.028035,r
4997869293954662400,24.699699,56308.299339,36.651812,0.030037,g


## Analysis

In [9]:
ens.head("object")

/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwa

,nobs_g,nobs_i,nobs_r,nobs_u,nobs_z,nobs_total,SDSS_NAME_dr16q_constant,PLATE_dr16q_constant,MJD_dr16q_constant,FIBERID_dr16q_constant,...,zFlags_ps1_otmo,yMeanPSFMag_ps1_otmo,yMeanPSFMagErr_ps1_otmo,yFlags_ps1_otmo,Norder_ps1_otmo,Dir_ps1_otmo,Npix_ps1_otmo,_DIST,Norder,Dir
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
4996744218186612736,18,0,33,1,0,52,b'024311.71-010928.3',7337.0,56662.0,360.0,...,114720.0,-999.000000,-999.000000,16416.0,5.0,0.0,4438.0,0.000015,5,0
4996744985459032064,74,6,84,3,0,167,b'024321.81-010812.1',6782.0,56572.0,285.0,...,115000.0,-999.000000,-999.000000,16416.0,5.0,0.0,4438.0,0.000010,5,0
4996747488372195328,26,4,29,2,0,61,b'024315.98-010433.7',4240.0,55455.0,202.0,...,115000.0,-999.000000,-999.000000,114720.0,5.0,0.0,4438.0,0.000017,5,0
4996749076859977728,103,11,100,8,0,222,b'024328.22-010352.7',6782.0,56572.0,298.0,...,115000.0,-999.000000,-999.000000,114720.0,5.0,0.0,4438.0,0.000005,5,0
4996749735944519680,266,22,282,6,10,586,b'024341.05-010243.3',4240.0,55455.0,178.0,...,115000.0,19.193501,0.039844,115000.0,5.0,0.0,4438.0,0.000016,5,0


In [11]:
arg_container = StructureFunctionArgumentContainer()
arg_container.band_to_calc = ["g","r"]
arg_container.combine = False

In [12]:
ens.prune(30).sf2(argument_container = arg_container)

/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/astro/users/brantd/.conda/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,lc_id,band,dt,sf2,1_sigma
0,4996744218186612736,g,217.494673,-0.004436,0.0
1,4996744218186612736,g,1037.952208,58.457161,0.0
2,4996744218186612736,r,14.969049,0.037458,0.0
3,4996744218186612736,r,115.255781,0.161024,0.0
4,4996744218186612736,r,389.490094,0.021264,0.0
...,...,...,...,...,...
238,4997869293954662400,r,3203.248735,245.387225,0.0
239,4997869293954662400,r,3269.334458,244.511635,0.0
240,4997869293954662400,r,3417.907312,246.691394,0.0
241,4997869293954662400,r,3612.610135,243.430806,0.0
